In [1]:
pip install lancedb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 66.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 52.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 7.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import tqdm
import torch
import lancedb
import numpy as np
import pandas as pd
import pyarrow as pa
import urllib.request
import torch.nn.functional as F

from sklearn.metrics import f1_score
from pydantic import BaseModel
from transformers import AutoImageProcessor, TimesformerForVideoClassification

In [3]:
model_name = "facebook/timesformer-base-finetuned-k600"
device = "cuda" if torch.cuda.is_available() else "cpu"
processor = AutoImageProcessor.from_pretrained(model_name)
model = TimesformerForVideoClassification.from_pretrained(model_name).to(device)

preprocessor_config.json:   0%|          | 0.00/412 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/34.4k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/487M [00:00<?, ?B/s]

In [4]:
# Функция для загрузки и обработки видео
def load_video(video_path, f = 0, frame_height=480, frame_width=480):
    cap = cv2.VideoCapture(video_path)
    frames = []
    ret, frame = cap.read()
    while ret:
            frame = cv2.resize(frame, (frame_width, frame_height))
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)   
            frame = frame[110:350, 50:430]
            frames.append(frame)
            ret, frame = cap.read()
    cap.release()
    idxs = np.linspace(0, len(frames) - 1, 16, dtype=int)
    frames = list(np.array(frames)[idxs])
    if f:
        frames = [cv2.flip(frame, 1) for frame in frames]
    return frames

# Функция для получения эмбеддинга видео
def make_embedding(video_path, model, processor, f = 0):
    frames = load_video(video_path, f)
    inputs = processor(images=frames, return_tensors="pt").to(device)
    del frames
    outputs = model(**inputs, output_hidden_states=True)
    del inputs
    return outputs.hidden_states[-1].squeeze(0).mean(dim=1)

In [18]:
# url = 'https://s3.ritm.media/yappy-db-duplicates/2fa37210-3c25-4a87-88f2-1242c2c8a699.mp4'


# file_Path = 'short.mp4'
# urllib.request.urlretrieve(url, file_Path)

# emb = make_embedding(file_Path, model, processor)
# print(emb.shape)

torch.Size([3137])


In [5]:
table = pd.read_csv("/kaggle/input/vseros/test.csv")

In [6]:
uri = "data/video-lancedb"
db = lancedb.connect(uri)

In [7]:
custom_schema = pa.schema([pa.field("vector", pa.list_(pa.float32(), 3137)),
                           pa.field("id", pa.string())
                           ])

tbl = db.create_table("emb_table", schema=custom_schema)

In [8]:
uuid, link = table[["uuid", "link"]].iloc[0]
url = link
file_Path = 'short.mp4'
urllib.request.urlretrieve(url, file_Path)
emb = make_embedding(file_Path, model, processor).cpu().detach().numpy()
data = [{"vector": emb, "id": uuid}]
tbl.add(data)

/opt/conda/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:142: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(value)


In [9]:
preds_flags = [False]
preds_uuid = ['']

In [10]:
for uuid, link in tqdm.tqdm(table[["uuid", "link"]].values[1:]):
    url = link
    file_Path = 'short.mp4'
    urllib.request.urlretrieve(url, file_Path)
    emb = make_embedding(file_Path, model, processor)
    numpy_emb = emb.cpu().detach().numpy()
    res = tbl.search(numpy_emb).limit(1).metric("cosine").to_pandas()
    id_vec = res.id.values[0]
    vec = torch.tensor(list(res.vector)).squeeze(0).to(device)
    cosine_similarity = F.cosine_similarity(emb, vec, dim=0).item()
    del emb
    if cosine_similarity < 0.4:
        mirror_emb = make_embedding(file_Path, model, processor, 1)
        mirror_cosine_similarity = F.cosine_similarity(mirror_emb, vec, dim=0).item()
        if mirror_cosine_similarity < 0.4:
            data = [{"vector": numpy_emb, "id": uuid}]
            tbl.add(data)
            preds_flags.append(False)
            preds_uuid.append('')
        else:
            preds_flags.append(True)
            preds_uuid.append(id_vec)
    else:
        preds_flags.append(True)
        preds_uuid.append(id_vec)

100%|██████████| 999/999 [2:24:30<00:00,  8.68s/it]  


In [11]:
table["is_duplicate"] = preds_flags
table["duplicate_for"] = preds_uuid

In [12]:
table = table.set_index('created')

In [13]:
table.to_csv('res.csv')